In [ ]:
import numpy as np
import torch
from torch import nn
from copy import deepcopy
import matplotlib.pyplot as plt
from scipy.stats import beta
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(12) # for reproducibility
torch.manual_seed(12) # for reproducibility
alpha_index = 0.1 # 0.9 of confiability (1-α)
awgn_situation = True # If True, the channel is AWGN

def lin2dB(x):
    return 10*np.log10(x) # Convert linear scale to dB

def dB2lin(x):
    return 10**(x/10) # Convert dB scale to linear


## 1 - 8APSK Constellation
def get8APSK():
    """Generate 8APSK constellation parameters.

    Returns:
        dict: Dictionary containing:
            - constellation: 8APSK constellation points
            - internal_radius: Inner ring radius
            - external_radius: Outer ring radius
            - gray_mapping: Gray mapping
            - bit_mapping: Bit mapping
            - normalized: Normalization flag
            - description: Constellation description
    """


    # Internal and external radius
    apsk8_internal_radius = 2.0 / torch.sqrt(2.0 + (1 + torch.sqrt(torch.tensor(3.0))) ** 2)  # ~0.65
    apsk8_external_radius = torch.sqrt(2.0 - apsk8_internal_radius ** 2)  # ~1.26

    # Making the mapping
    vMapping = torch.cat(
                                [torch.tensor([+1.+1.j, -1.+1.j, +1.-1.j, -1.-1.j], dtype=torch.complex64) * apsk8_internal_radius/torch.tensor(2.0).sqrt(),
                                 torch.tensor([+1.,     +1.j,    -1,      -1.j   ], dtype=torch.complex64) * apsk8_external_radius]
                                 ) # Shifted by 45º

    gray_mapping = {
        0: 0,  # 000
        1: 1,  # 001
        2: 3,  # 011
        3: 2,  # 010
        4: 6,  # 110
        5: 7,  # 111
        6: 5,  # 101
        7: 4   # 100
    }

    bit_mapping = {
        0: [0,0,0], 1: [0,0,1], 3: [0,1,1], 2: [0,1,0],
        6: [1,1,0], 7: [1,1,1], 5: [1,0,1], 4: [1,0,0]
    }

    return {
        'constellation': vMapping,
        'internal_radius': apsk8_internal_radius,
        'external_radius': apsk8_external_radius,
        'gray_mapping': gray_mapping,
        'bit_mapping': bit_mapping,
        'normalized': True,
        'description': '8APSK constellation with Gray coding',
        'order': 8  # Fixed order for 8APSK
    }

def modulate_8apsk(bits, constellation_params=None):
    """Modulate bits to 8APSK symbols using Gray mapping.

    Args:
        bits: Bit sequence for modulation (shape [N,3] or [3N])
        constellation_params: Constellation parameters (optional)

    Returns:
        tuple: (modulated symbols, Gray-mapped indices)
    """


    if constellation_params is None:
        constellation_params = get8APSK()

    # Converte bits para tensor de forma segura
    if isinstance(bits, torch.Tensor):
        bits = bits.detach().clone().float()
    else:
        bits = torch.tensor(bits, dtype=torch.float32)

    # Garante shape [N, 3]
    if bits.dim() == 1:
        bits = bits.view(-1, 3)

    # Converte bits para índices inteiros (0-7)
    bit_indices = (bits @ torch.tensor([4, 2, 1], dtype=torch.float32)).long()

    # Aplica mapeamento Gray - versão corrigida
    gray_indices = torch.tensor([constellation_params['gray_mapping'][idx.item()] for idx in bit_indices])

    # Modulação
    symbols = constellation_params['constellation'][gray_indices]

    return symbols, gray_indices


def generate_8apsk_samples(num_samples, pattern=0, constellation_params=None):
    """Generates 8APSK samples in different patterns.

    Args:
        num_samples: Number of samples to generate
        pattern: 0=cyclic sequence, 1=blocks, 2=random
        constellation_params: Constellation parameters (optional)

    Returns:
        tuple: (complex symbols, symbol indices)
    """
    if constellation_params is None:
        constellation_params = get8APSK()

    order = 8  # Ordem fixa para 8APSK

    # Gera os índices dos símbolos
    if pattern == 0:  # Sequência cíclica 0,1,2,...,7,0,1,...
        symbol_indices = torch.arange(0, num_samples, dtype=torch.long) % order
    elif pattern == 1:  # Blocos 0,0,...,0,1,1,...,1,...
        symbol_indices = (torch.arange(0, num_samples, dtype=torch.long) // order) % order
    else:  # Aleatório balanceado
        symbol_indices = torch.randperm(num_samples) % order

    # Modula os símbolos
    symbols = constellation_params['constellation'][symbol_indices]

    return symbols, symbol_indices

def slicer_demodulation(rx_symbols, constellation_params):
    """Performs hard decision demodulation for 8APSK constellation.

    Args:
        rx_symbols: Received complex symbols
        constellation_params: Dictionary containing constellation parameters

    Returns:
        torch.Tensor: Indices of the closest constellation points
    """


    constellation = constellation_params['constellation']
    distances = torch.abs(rx_symbols.unsqueeze(1) - constellation)
    return torch.argmin(distances, dim=1)


def plot_decision_borders(constellation_params):
    """Plots the 8APSK constellation with decision boundaries.

    Args:
        constellation_params: Dictionary containing constellation parameters
                            including 'constellation', 'internal_radius',
                            and 'external_radius'

    Returns:
        None: Displays the plot using matplotlib
    """

    # Plota os pontos da constelação
    constellation = constellation_params['constellation']
    plt.scatter(constellation.real, constellation.imag, c='b', marker='o')

    # Plota os círculos que separam os anéis
    r_in = constellation_params['internal_radius']
    r_out = constellation_params['external_radius']
    circle_in = plt.Circle((0,0), r_in, fill=False, linestyle='--', color='k')
    circle_out = plt.Circle((0,0), r_out, fill=False, linestyle='--', color='k')
    plt.gca().add_artist(circle_in)
    plt.gca().add_artist(circle_out)

    plt.grid(True)
    plt.axis('equal')
    plt.title('8APSK Constellation')



## 2 - System Model
def channel_states():
    """
    Simulates and returns a new channel state dictionary.

    Returns:
        dict: Dictionary containing channel state parameters:
              'tx_amp_imbalance_factor', 'tx_phase_imbalance_factor', 'ch_mult_factor'.
    """
    # Generate the channel multiplication factor
    ch_mult_factor = torch.randn(1, dtype=torch.cfloat) # Corresponds to ψ (psi) in the article

    # Create a beta distribution for the transmitter imperfection factors
    # CORREÇÃO: Especifique dtype=torch.float ou torch.double para os tensores
    rand_beta_dist_instance = torch.distributions.Beta(torch.tensor(5.0), torch.tensor(2.0)) # Ou use .float() ou .double()

    # If AWGN situation is forced, remove amplitude variation from fading
    if awgn_situation:
        ch_mult_factor /= ch_mult_factor.abs()

    # Sample transmitter imperfection factors using the beta distribution instance
    tx_amplitude_imbalance_factor = rand_beta_dist_instance.sample() * 0.15 # Corresponds to ϵ (epsilon) in the article
    tx_phase_imbalance_factor = rand_beta_dist_instance.sample() * 15 * np.pi / 180 # Corresponds to δ (delta) in the article

    # Return the channel state dictionary, "c" in the article
    d_channel_state = {
        'tx_amp_imbalance_factor': tx_amplitude_imbalance_factor,
        'tx_phase_imbalance_factor': tx_phase_imbalance_factor,
        'ch_mult_factor': ch_mult_factor
    }

    return d_channel_state

def simulate_channel_application(num_samples, b_enforce_pattern, b_noise_free,
                                d_setting, channel_state, mod_constellation_params):
    """
    Simulates a channel step for demodulation, applying imperfections and noise.

    Args:
        num_samples (int): Number of samples to generate.
        b_enforce_pattern (bool): If True, uses a fixed pattern for TX symbol generation.
        b_noise_free (bool): If True, no noise is added.
        d_setting (dict): Configuration dictionary, containing 'snr_dB'.
        channel_state (dict): Dictionary containing channel state parameters (ψ, ϵ, δ).
        mod_constellation_params (dict): Modulation constellation parameters (from get_8apsk_constellation_params).

    Returns:
        tuple: (rx_real_iq, tx_sym_uint)
               rx_real_iq (torch.Tensor): Received signals (real and imaginary parts separated).
               tx_sym_uint (torch.Tensor): Original transmitted symbols (integer indices).
    """
    # Define the symbol generation pattern
    pattern = 0 if b_enforce_pattern else -1

    # 1. Generate TX Symbols
    tx_iq, tx_sym_uint = generate_8apsk_samples(num_samples, pattern, mod_constellation_params)

    # 2. Get channel state parameters
    epsilon = channel_state['tx_amp_imbalance_factor'] # ϵ (epsilon) from the article
    ch_mult_factor = channel_state['ch_mult_factor']   # Complex channel factor (contains ψ)
    delta = channel_state['tx_phase_imbalance_factor'] # δ (delta) from the article

    cos_delta = torch.cos(delta)
    sin_delta = torch.sin(delta)

    # 3. Apply Transmitter Hardware Imperfections (I/Q Imbalance and Phase Rotation, Eq. 28 from the article) [cite: 248]

    tx_distorted_real = (1 + epsilon) * (cos_delta * tx_iq.real - sin_delta * tx_iq.imag)
    tx_distorted_imag = (1 - epsilon) * (cos_delta * tx_iq.imag - sin_delta * tx_iq.real)

    tx_distorted = tx_distorted_real + 1j * tx_distorted_imag # This is the f_IQ function from the article [cite: 246, 248]

    # 4. Apply Channel Effect (Complex Multiplication) [cite: 246]
    tx_rayleighed = tx_distorted * ch_mult_factor

    # 5. Add Noise (AWGN) [cite: 246]
    rx_iq = tx_rayleighed # Initialize with the signal before noise addition
    if not b_noise_free: # If it's not a noise-free situation
        snr_lin = dB2lin(d_setting['snr_dB']) # Calculate linear SNR from d_setting
        noise_std_dev = np.sqrt(0.5 / snr_lin)
        noise = noise_std_dev * (torch.randn(num_samples, dtype=torch.float64) + 1j * torch.randn(num_samples, dtype=torch.float64))
        rx_iq += noise # Add noise

    # 6. Convert the received signal to real format (I and Q separated)
    # The neural network expects inputs as real tensors [num_samples, 2]
    rx_real_iq = torch.stack([rx_iq.real, rx_iq.imag], dim=1).type(torch.float64)

    return rx_real_iq, tx_sym_uint



## 3 - Data Set Preparation
def leave_one_out_data(x_input: torch.Tensor, y_output: torch.Tensor, index: int): #-> tuple[torch.Tensor, torch.Tensor]:
    """
    Returns a new dataset with one sample removed (leave-one-out - LOO). Used to CV-CP

    Args:
        x_input (torch.Tensor): Input features tensor.
        y_output (torch.Tensor): Output labels tensor.
        index (int): Index of the sample to be removed.

    Returns:
        tuple[torch.Tensor, torch.Tensor]: (X_loo, y_loo) - Tensors representing the dataset with one sample left out.
    """
    N_len = len(y_output)

    # Input validation (optional, but good practice)
    if not (0 <= index < N_len):
        raise IndexError(f"Index {index} is out of bounds for dataset of size {N_len}.")

    # Create indices for all samples except the one at 'index'
    indices_loo = np.concatenate((np.arange(0, index), np.arange(index + 1, N_len)), axis=0)

    # Convert numpy array of indices to a PyTorch tensor for advanced indexing
    indices_loo_tensor = torch.from_numpy(indices_loo).long()

    # Use the generated indices to select elements from X and y
    X_loo = x_input[indices_loo_tensor, :]
    y_loo = y_output[indices_loo_tensor]

    return X_loo, y_loo

def leave_fold_out_data(x_input: torch.Tensor, y_output: torch.Tensor, fold_index: int, num_folds: int): #-> tuple[torch.Tensor, torch.Tensor]:
    """
    Returns a new dataset with one fold removed (leave-fold-out - LFO). Used to K-CV-CP

    Args:
        x_input (torch.Tensor): Input features tensor.
        y_output (torch.Tensor): Output labels tensor.
        fold_index (int): Index of the fold to be removed (0-indexed).
        num_folds (int): Total number of folds.

    Returns:
        tuple[torch.Tensor, torch.Tensor]: (X_lfo, y_lfo) - Tensors representing the dataset with one fold left out.
    """
    N_len = len(y_output)
    # Calculate the number of samples per fold. Rounding is consistent with original code.
    N_over_K = round(N_len / num_folds)

    if N_len % num_folds != 0:
        print(f"Warning: Dataset size ({N_len}) is not perfectly divisible by num_folds ({num_folds}).")

    # Input validation
    if not (0 <= fold_index < num_folds):
        raise IndexError(f"Fold index {fold_index} is out of bounds for {num_folds} folds.")

    # Calculate start and end indices of the fold to be excluded
    fold_start_idx = fold_index * N_over_K
    fold_end_idx = (fold_index + 1) * N_over_K

    # Concatenate indices for the segments before and after the excluded fold
    indices_lfo = np.concatenate((np.arange(0, fold_start_idx), np.arange(fold_end_idx, N_len)), axis=0)

    # Convert numpy array of indices to a PyTorch tensor for advanced indexing
    indices_lfo_tensor = torch.from_numpy(indices_lfo).long()

    # Select elements using the generated indices
    X_lfo = x_input[indices_lfo_tensor, :]
    y_lfo = y_output[indices_lfo_tensor]

    return X_lfo, y_lfo


def split_data_into_subsets(x_input: torch.Tensor, y_output: torch.Tensor, N_samples_first_subset: int, shuffle: bool = True):# -> tuple[tuple[torch.Tensor, torch.Tensor], tuple[torch.Tensor, torch.Tensor]]:
    """
    Splits the dataset into two subsets (e.g., training and validation). Used to VB-CP

    Args:
        x_input (torch.Tensor): Input features tensor.
        y_output (torch.Tensor): Output labels tensor.
        N_samples_first_subset (int): Number of samples for the first subset.
        shuffle (bool, optional): If True, shuffles the data before splitting. Defaults to True.

    Returns:
        tuple[tuple[torch.Tensor, torch.Tensor], tuple[torch.Tensor, torch.Tensor]]:
            A tuple containing two (X, y) tuples: ((X0, y0), (X1, y1)),
            representing the training and validation subsets respectively.
    """
    N_len = len(y_output)

    # Handle edge cases where one subset is empty or all data goes into one subset
    if N_samples_first_subset == 0: # All data in second subset
        return ( (torch.empty(0, x_input.shape[1], dtype=x_input.dtype), torch.empty(0, dtype=y_output.dtype)), (x_input, y_output) )
    elif N_samples_first_subset == N_len: # All data in first subset
        return ( (x_input, y_output), (torch.empty(0, x_input.shape[1], dtype=x_input.dtype), torch.empty(0, dtype=y_output.dtype)) )

    # Generate permutation indices
    if shuffle:
        perm = torch.randperm(N_len)
    else:
        perm = torch.arange(0, N_len)

    # Split X and y using the generated permutation indices
    X0 = x_input[perm[:N_samples_first_subset], :]
    y0 = y_output[perm[:N_samples_first_subset]]

    X1 = x_input[perm[N_samples_first_subset:], :] # Corrected slicing for N1 (rest of the data)
    y1 = y_output[perm[N_samples_first_subset:]] # Corrected slicing for N1 (rest of the data)

    return ( (X0, y0), (X1, y1) )



## 4 - Neural Network Models
class FcReluDnn(nn.Module):
    """
    Fully-Connected ReLU Deep Neural Network.
    This class defines the neural network architecture.
    """
    def __init__(self, vLayers: list):
        """
        Constructor for the FcReluDnn model.

        Args:
            vLayers (list): A list of integers defining the number of neurons in each layer.
                            Example: [input_dim, hidden1_dim, hidden2_dim, ..., output_dim].
        """
        super(FcReluDnn, self).__init__()

        self.hidden = nn.ModuleList() # Use ModuleList to store linear layers

        # Create linear layers with ReLU activation for hidden layers
        # and a final linear layer for the output (no activation here, softmax applied later).
        for l_idx, (input_size, output_size) in enumerate(zip(vLayers, vLayers[1:])):
            # All layers use torch.float64 as specified in the original code for Hessian calculations
            linear_layer = nn.Linear(input_size, output_size, dtype=torch.float64)
            self.hidden.append(linear_layer)

    def forward(self, activation: torch.Tensor): # -> torch.Tensor:
        """
        Forward pass through the neural network.

        Args:
            activation (torch.Tensor): Input tensor to the network.

        Returns:
            torch.Tensor: Output tensor (logits before softmax).
        """
        L = len(self.hidden) # Number of layers

        for l_idx, linear_transform in enumerate(self.hidden):
            activation = linear_transform(activation)
            # Apply ReLU activation for all hidden layers (not the last output layer)
            if l_idx < L - 1:
                activation = torch.nn.functional.relu(activation)
        return activation

    def num_parameters(self): # -> int:
        """
        Calculates the total number of trainable parameters in the model.

        Returns:
            int: Total number of parameters.
        """
        return sum(torch.numel(w) for w in self.parameters())


class FcReluDnn_external(nn.Module):
    """
    Fully-Connected ReLU Deep Neural Network designed to operate with externally provided parameters.
    This is used when model parameters are managed and passed explicitly (e.g., for Hessian calculations).
    """
    def __init__(self):
        """
        Constructor for the FcReluDnn_external model.
        It does not initialize its own nn.Linear layers, as parameters are external.
        """
        super(FcReluDnn_external, self).__init__()
        # Note: No need to initialize nn.Linear layers here since parameters are provided externally.
        # This module will be used to apply linear transformations and activations given external weights/biases.

    def forward(self, net_in: torch.Tensor, net_params: list[torch.Tensor]): # -> torch.Tensor:
        """
        Forward pass using externally provided network parameters.

        Args:
            net_in (torch.Tensor): Input tensor to the network.
            net_params (list[torch.Tensor]): A list of tensors representing the weights and biases of all layers.
                                            Assumed to be ordered as [weight1, bias1, weight2, bias2, ...].

        Returns:
            torch.Tensor: Output tensor (logits before softmax).
        """
        # Calculate number of layers from the parameter list (each layer has weight and bias)
        L = len(net_params) // 2

        for ll in range(L):
            curr_layer_weight = net_params[2 * ll]
            curr_layer_bias = net_params[2 * ll + 1]

            # Apply linear transformation using functional API
            net_in = torch.nn.functional.linear(net_in, curr_layer_weight, curr_layer_bias)

            # Apply ReLU activation for all hidden layers (not the last output layer)
            if ll < L - 1:
                net_in = torch.nn.functional.relu(net_in)
        return net_in



## 5 - Training and Evaluation Functions
def fitting_erm_ml__gd(
    model: nn.Module, D_X: torch.Tensor,  D_y: torch.Tensor, D_te_X: torch.Tensor, D_te_y: torch.Tensor, gd_init_sd: dict, gd_lr: float, gd_num_iters: int):# -> tuple[torch.Tensor, torch.Tensor]:
    """
    Trains a neural network model using Empirical Risk Minimization (ERM)
    with Maximum Likelihood (ML) objective via full-batch Gradient Descent.

    Args:
        model (nn.Module): The neural network model instance to be trained. Its parameters
                           will be updated during training.
        D_X (torch.Tensor): Features (X) of the training dataset.
        D_y (torch.Tensor): Labels (y) of the training dataset.
        D_te_X (torch.Tensor): Features (X) of the test dataset (used only for validation loss tracking).
        D_te_y (torch.Tensor): Labels (y) of the test dataset (used only for validation loss tracking).
        gd_init_sd (dict): Initial state dictionary of the model's parameters to reset the model before training.
        gd_lr (float): Learning rate for Gradient Descent.
        gd_num_iters (int): Number of Gradient Descent iterations.

    Returns:
        tuple[torch.Tensor, torch.Tensor]:
            - v_loss_tr (torch.Tensor): Tensor of training loss values across iterations.
            - v_loss_te (torch.Tensor): Tensor of test loss values across iterations.
    """
    # Reset model to its initial state before starting training
    model.load_state_dict(gd_init_sd)

    # Define the loss criterion for classification (Cross Entropy Loss)
    criterion = nn.CrossEntropyLoss()

    # Tensors to store loss history during training
    v_loss_tr = torch.zeros(gd_num_iters, dtype=torch.float)
    v_loss_te = torch.zeros(gd_num_iters, dtype=torch.float)

    for i_iter in range(gd_num_iters):
        # Evaluate test loss at current iteration (detached from graph to prevent gradient calculation)
        v_loss_te[i_iter] = criterion(model(D_te_X), D_te_y).detach()

        # Zero out gradients for all model parameters before performing a new backward pass.
        # This is a manual equivalent of `optimizer.zero_grad()`.
        for w in model.parameters():
            if w.grad is not None:
                w.grad.data.zero_() # Set existing gradients to zero
            else:
                w.grad = torch.zeros_like(w.data) # Initialize if grad is None (first iteration)

        # Perform forward pass on the full training dataset and calculate the training loss
        # (This implements full-batch Gradient Descent)
        loss = criterion(model(D_X), D_y)

        # Perform backward pass to compute gradients of the loss with respect to model parameters
        loss.backward()

        # Update model parameters using the computed gradients
        # This is a manual equivalent of `optimizer.step()`.
        for w in model.parameters():
            w.data -= gd_lr * w.grad.data # Parameter update rule for GD

        # Store training loss for current iteration (detached from graph)
        v_loss_tr[i_iter] = loss.detach()

    return v_loss_tr, v_loss_te


def eval_hessian(loss_grad: tuple[torch.Tensor, ...], model: nn.Module): #-> torch.Tensor:
    """
    Evaluates the full Hessian matrix from a flat list of first-order gradients.
    This function is based on a common PyTorch recipe for computing the Hessian.

    Args:
        loss_grad (tuple[torch.Tensor, ...]): Tuple of first-order gradients (obtained with create_graph=True).
                                             These gradients are computed with respect to model.parameters().
        model (nn.Module): The model instance whose parameters' Hessian is being computed.

    Returns:
        torch.Tensor: The reconstructed square Hessian matrix.
    """
    # Flatten all first-order gradients into a single vector
    g_vector = torch.cat([g.contiguous().view(-1) for g in loss_grad])

    l = g_vector.size(0) # Total number of parameters (length of the flattened gradient vector)
    hessian = torch.zeros((l, l), dtype=torch.float64) # Initialize the Hessian matrix (l x l) with zeros

    # Compute second-order gradients for each element in the flattened first-order gradient vector
    for idx in range(l):
        # Compute gradients of g_vector[idx] (a single scalar element of the first-order gradient)
        # with respect to all parameters of the model.
        # `create_graph=True` is crucial here because we need to compute gradients of these
        # second-order gradients later if necessary (though not in this specific function's return).
        grad2rd = torch.autograd.grad(g_vector[idx], model.parameters(), create_graph=True)

        # Flatten these second-order gradients (which are also a tuple of tensors)
        # and assign them as a row in the Hessian matrix.
        hessian[idx] = torch.cat([g.contiguous().view(-1) for g in grad2rd])

    return hessian


def fitting_erm_map_gd(model: nn.Module, D_X_training: torch.Tensor, D_y_training: torch.Tensor, D_test_X: torch.Tensor, D_test_y: torch.Tensor, gd_init_sd: dict, gd_lr: float, gd_num_iters: int, gamma: float, ensemble_size: int, compute_hessian: int, lmc_burn_in: int, lmc_lr_init: float, lmc_lr_decaying: float, compute_bays: bool): #-> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Trains a neural network model using Empirical Risk Minimization (ERM)
    with Maximum A Posteriori (MAP) objective via full-batch Gradient Descent.
    Also generates Bayesian ensembles using Langevin Monte Carlo (LMC)
    and Hessian/Fisher Information Matrix (FIM) approximations.

    Args:
        model (nn.Module): The neural network model instance for MAP training. Its parameters
                           will be updated.
        D_X_training (torch.Tensor): Features (X) of the training dataset.
        D_y_training (torch.Tensor): Labels (y) of the training dataset.
        D_test_X (torch.Tensor): Features (X) of the test dataset (for evaluation only).
        D_test_y (torch.Tensor): Labels (y) of the test dataset (for evaluation only).
        gd_init_sd (dict): Initial state dictionary for the model.
        gd_lr (float): Learning rate for MAP/GD optimization.
        gd_num_iters (int): Number of iterations for MAP optimization.
        gamma (float): L2 regularization coefficient for MAP (prior precision).
        ensemble_size (int): Number of model samples for Bayesian ensembles.
        compute_hessian (int): Flag for Hessian computation (0: none, 1: model-based, 2: grad(grad)).
        lmc_burn_in (int): Number of initial samples to discard in LMC.
        lmc_lr_init (float): Initial learning rate for LMC.
        lmc_lr_decaying (float): Decay factor for LMC learning rate.
        compute_bays (bool): Flag to enable Bayesian ensemble generation.

    Returns:
        tuple: (v_loss_tr_map, v_loss_te_map, v_lmc_loss_tr, v_lmc_loss_te,
                ensemble_vectors_hes, ensemble_vectors_fim, ensemble_vectors_lmc)
            - v_loss_tr_map (torch.Tensor): Training loss history for MAP.
            - v_loss_te_map (torch.Tensor): Test loss history for MAP.
            - v_lmc_loss_tr (torch.Tensor): Training loss history for LMC.
            - v_lmc_loss_te (torch.Tensor): Test loss history for LMC.
            - ensemble_vectors_hes (torch.Tensor): Ensemble of model parameters from Hessian approximation.
            - ensemble_vectors_fim (torch.Tensor): Ensemble of model parameters from FIM approximation.
            - ensemble_vectors_lmc (torch.Tensor): Ensemble of model parameters from LMC.
    """
    # 1. Train MAP solution
    model.load_state_dict(gd_init_sd) # Reset model to initial state
    lmc_model = deepcopy(model) # Model for Langevin Monte Carlo (starts from same initial state as MAP)
    ext_model = FcReluDnn_external() # Instance of the external model class, used for compute_hessian == 1
    criterion = nn.CrossEntropyLoss() # Loss function for both MAP and LMC
    N = len(D_y_training) # Number of training samples (from training data labels)

    v_loss_tr_map = torch.zeros(gd_num_iters, dtype=torch.float64)
    v_loss_te_map = torch.zeros(gd_num_iters, dtype=torch.float64)

    for i_iter in range(gd_num_iters):
        v_loss_te_map[i_iter] = criterion(model(D_test_X), D_test_y).detach() # Evaluate test loss for MAP model

        # Zero out gradients for all model parameters before backward pass
        for w in model.parameters():
            if w.grad is not None: w.grad.data.zero_()
            else: w.grad = torch.zeros_like(w.data) # Initialize if None

        loss = criterion(model(D_X_training), D_y_training) # Calculate training loss (likelihood term)
        loss.backward() # Compute gradients

        # Update parameters with L2 regularization term (MAP update rule)
        for w in model.parameters():
            w.data -= gd_lr * (w.grad.data + gamma / N * w.data)
        v_loss_tr_map[i_iter] = loss.detach()

    # Get the final MAP solution parameters as a flattened vector
    v_phi_map = torch.nn.utils.parameters_to_vector(model.parameters()).double()

    # Initialize ensemble vectors with the MAP solution (these will be updated or replaced later)
    # .detach().clone() to create independent copies
    ensemble_vectors_lmc = v_phi_map.detach().clone().repeat(ensemble_size, 1)
    ensemble_vectors_hes = deepcopy(ensemble_vectors_lmc) # Will be overwritten if compute_hessian != 0
    ensemble_vectors_fim = deepcopy(ensemble_vectors_lmc) # Will be overwritten if compute_bays is False or compute_hessian is 0 and FIM is not computed

    # --- Bayesian Ensemble Generation ---
    if compute_bays: # Flag to enable Bayesian computations
        # 2. Langevin Monte Carlo (LMC) approach for creating an ensemble
        lmc_num_iters = lmc_burn_in + ensemble_size - 1 # Total LMC iterations

        # Learning rate scheduling for LMC (decaying or constant)
        if lmc_lr_decaying > 1.0:
            lmc_lr_last = lmc_lr_init / lmc_lr_decaying
            lmc_lr_gamma = 0.55 # Fixed gamma for decay (from original code)
            # 'b' and 'a' parameters define the decaying function: lr_a * (lr_b + iter)**(-lr_gamma)
            lmc_lr_b = (lmc_num_iters - 1) / (((lmc_lr_init / lmc_lr_last)**(1 / lmc_lr_gamma)) - 1)
            lmc_lr_a = lmc_lr_init * (lmc_lr_b**lmc_lr_gamma)
        else: # Constant learning rate
            lmc_lr_gamma = 0
            lmc_lr_b = 0
            lmc_lr_a = lmc_lr_init

        v_lmc_loss_tr = torch.zeros(lmc_num_iters, dtype=torch.float)
        v_lmc_loss_te = torch.zeros(lmc_num_iters, dtype=torch.float)
        lmc_temperature = 20.0 # Langevin MC temperature (from original code)

        for i_iter in range(lmc_num_iters):
            lmc_lr = lmc_lr_a * (lmc_lr_b + i_iter)**(-lmc_lr_gamma) # Calculate current LMC learning rate
            v_lmc_loss_te[i_iter] = criterion(lmc_model(D_test_X), D_test_y).detach() # Evaluate test loss for LMC model

            # Zero gradients for LMC model parameters
            for w in lmc_model.parameters():
                if w.grad is not None: w.grad.data.zero_()
                else: w.grad = torch.zeros_like(w.data)

            loss = criterion(lmc_model(D_X_training), D_y_training) # Calculate training loss for LMC
            loss.backward() # Compute gradients for LMC

            for w in lmc_model.parameters():
                # LMC update rule:
                # Gradient from likelihood + gradient from prior (L2 reg) + Gaussian noise
                lmc_update_term = - lmc_lr * (w.grad.data + gamma / N * w.data)
                # Noise is added only after the burn-in period (original code had a commented out `if 0:` condition
                # implying noise *always* added or *never* during burn-in; following the `else` block as the intended logic)
                if (i_iter + 1 >= lmc_burn_in):
                    # Add Gaussian noise scaled by learning rate, N, and temperature
                    lmc_update_term += np.sqrt(2 * lmc_lr / (N * lmc_temperature)) * torch.randn_like(w.data)
                w.data += lmc_update_term

            # After the burn-in period, save the model parameters to form the ensemble
            if (i_iter + 1 >= lmc_burn_in):
                r = i_iter + 1 - lmc_burn_in # Index for the ensemble vector
                if r < ensemble_size: # Ensure we only save up to ensemble_size samples
                    ensemble_vectors_lmc[r, :] = torch.nn.utils.parameters_to_vector(lmc_model.parameters())
            v_lmc_loss_tr[i_iter] = loss.detach()

        # 3. Hessian and Fisher Information Matrix (FIM) approaches for Bayesian ensemble
        # These methods approximate the posterior distribution of parameters as a Gaussian.
        # This part can be computationally intensive.
        if compute_hessian == 1: # Model-based Hessian using FcReluDnn_external
            model_for_hessian = deepcopy(model) # Start from the trained MAP model
            # Define a nested function for loss calculation used by torch.autograd.functional.hessian
            # This function needs to accept parameters as a tuple which FcReluDnn_external will use
            # to compute the forward pass.
            def loss_for_hessian_func(*in_params):
                # Using ext_model to compute loss given external parameters
                return criterion(ext_model(D_X_training, list(in_params)), D_y_training)

            # Compute the full Hessian matrix using torch.autograd.functional.hessian
            # This returns a nested tuple of tensors
            t_hessian_nested = torch.autograd.functional.hessian(
                loss_for_hessian_func,
                tuple(model_for_hessian.parameters()) # Pass current parameters as a tuple
            )

            # Reconstruct the full Hessian matrix from the nested tuple structure
            # This part flattens the nested structure into a single square matrix
            m_hessian = torch.cat([torch.cat([block.reshape(-1) for block in row], dim=0).unsqueeze(0) for row in t_hessian_nested], dim=0)
            m_hessian = m_hessian.reshape(v_phi_map.numel(), v_phi_map.numel()) # Reshape to square matrix
            m_hessian = 0.5 * (m_hessian + m_hessian.T) # Symmetrize to remove numerical asymmetry

        elif compute_hessian == 2: # Gradient of Gradient Hessian computation
            model_for_hessian = deepcopy(model) # Start from the trained MAP model
            # Zero gradients for the model used in Hessian calculation
            for w in model_for_hessian.parameters():
                if w.grad is not None: w.grad.data.zero_()
                else: w.grad = torch.zeros_like(w.data)

            loss_for_hessian_grad2 = criterion(model_for_hessian(D_X_training), D_y_training)
            # Compute first-order gradients with create_graph=True to allow for second-order gradient calculation
            loss_grad = torch.autograd.grad(
                loss_for_hessian_grad2,
                model_for_hessian.parameters(),
                create_graph=True # Essential for computing second-order gradients
            )
            # Compute the full Hessian matrix using the external eval_hessian helper function
            m_hessian = eval_hessian(loss_grad, model_for_hessian)
            m_hessian = 0.5 * (m_hessian + m_hessian.T) # Symmetrize

        # 4. Empirical Fisher Information Matrix (FIM) approximation
        # FIM is approximated by the average outer product of gradients for each sample.
        m_fim_D = torch.zeros((v_phi_map.numel(), v_phi_map.numel()), dtype=torch.float64)

        for x_sample, y_sample in zip(D_X_training, D_y_training): # Loop over individual training samples
            # Zero gradients for each sample's gradient calculation
            for w in model.parameters():
                if w.grad is not None: w.grad.data.zero_()
                else: w.grad = torch.zeros_like(w.data)

            # Compute loss for one sample (reshaping x_sample to (1,-1) for model input, y_sample to (-1) for criterion)
            loss_per_sample = criterion(model(x_sample.view(1, -1)), y_sample.view(-1))

            # Compute gradients for this single sample
            # retain_graph=True is needed because `model` is used across samples in this loop.
            v_grad_per_sample = torch.nn.utils.parameters_to_vector(
                torch.autograd.grad(loss_per_sample, model.parameters(), retain_graph=True)
            )
            # Add outer product of the gradient vector to FIM
            m_fim_D += torch.mm(v_grad_per_sample.view(-1, 1), v_grad_per_sample.view(1, -1))
        m_fim_D /= N # Normalize FIM by number of samples

        # 5. Form Ensembles from Hessian and FIM approximations (assuming Gaussian posterior)
        # This step approximates the posterior distribution of parameters as a multivariate Gaussian.
        # The mean is the MAP solution (v_phi_map), and the precision matrix is derived from Hessian/FIM.

        if compute_hessian == 0: # If compute_hessian is 0, assume infinite precision (delta function prior, no ensemble from Hessian)
            # This effectively makes mvn_hes sample the same MAP point repeatedly.
            m_precision_map_hes = torch.diag(torch.full([v_phi_map.numel()], np.inf, dtype=torch.float64))
            ensemble_vectors_hes = v_phi_map.detach().clone().repeat(ensemble_size, 1) # Simply repeat MAP solution
        else:
            # Precision matrix derived from Hessian: (gamma * Identity + N * Hessian)
            m_precision_map_hes = (gamma * torch.eye(v_phi_map.numel(), dtype=torch.float64) + N * m_hessian)
            # Create a MultivariateNormal distribution and sample ensemble vectors
            mvn_hes = torch.distributions.multivariate_normal.MultivariateNormal(
                loc=v_phi_map, # Mean vector is the MAP solution
                precision_matrix=m_precision_map_hes # Precision matrix
            )
            ensemble_vectors_hes = mvn_hes.sample(sample_shape=[ensemble_size])

        # Precision matrix derived from FIM: (gamma * Identity + N * FIM)
        m_precision_map_fim = (gamma * torch.eye(v_phi_map.numel(), dtype=torch.float64) + N * m_fim_D)
        # Create a MultivariateNormal distribution and sample ensemble vectors
        mvn_fim = torch.distributions.multivariate_normal.MultivariateNormal(
            loc=v_phi_map,
            precision_matrix=m_precision_map_fim
        )
        ensemble_vectors_fim = mvn_fim.sample(sample_shape=[ensemble_size])

    # Return all loss histories and ensemble vectors
    return v_loss_tr_map, v_loss_te_map, v_lmc_loss_tr, v_lmc_loss_te, ensemble_vectors_hes, ensemble_vectors_fim, ensemble_vectors_lmc


## 6 - Prediction and Nonconformity Score

def ensemble_predict(X: torch.Tensor, ensemble_vectors: torch.Tensor, model: nn.Module): # -> torch.Tensor:
    """
    Performs ensemble prediction by averaging softmax outputs from multiple models.
    Each model's parameters are loaded from `ensemble_vectors`.

    Args:
        X (torch.Tensor): Input features for prediction.
        ensemble_vectors (torch.Tensor): A tensor where each row is a flattened vector of model parameters.
                                         Shape: [num_ensemble_models, total_parameters].
        model (nn.Module): An instance of the neural network model (e.g., FcReluDnn). This model's
                           parameters will be temporarily loaded from `ensemble_vectors` for each prediction.

    Returns:
        torch.Tensor: Averaged softmax probabilities across the ensemble.
    """
    R = ensemble_vectors.size(dim=0) # Number of models in the ensemble

    m_prob_y = None # Initialize aggregated probabilities

    for r in range(R):
        # Load parameters for the current ensemble model
        torch.nn.utils.vector_to_parameters(ensemble_vectors[r, :], model.parameters())

        # Compute softmax probabilities for the current model
        current_prob_y = torch.nn.functional.softmax(model(X), dim=1)

        if r == 0:
            m_prob_y = current_prob_y
        else:
            m_prob_y += current_prob_y

    m_prob_y /= R # Average probabilities across the ensemble
    return m_prob_y

def nonconformity_frq(X: torch.Tensor, y: torch.Tensor, model: nn.Module): # -> torch.Tensor:
    """
    Calculates nonconformity scores for Frequentist (ML/MAP) predictions.
    Score is -log(softmax_probability_of_true_label). (Eq. 15 in main article)

    Args:
        X (torch.Tensor): Input features.
        y (torch.Tensor): True labels.
        model (nn.Module): Trained neural network model.

    Returns:
        torch.Tensor: Vector of nonconformity scores.
    """
    # Get softmax probabilities from the model
    m_prob_y = torch.nn.functional.softmax(model(X), dim=1)

    # Select probabilities corresponding to the true labels
    # torch.arange(len(y)) creates indices for rows, y selects the column (true label index)
    true_label_probs = m_prob_y[torch.arange(len(y)), y]

    # Calculate nonconformity score: -log(P(true_label)) (Eq. 15 in main article)
    return -torch.log(true_label_probs)

def nonconformity_bay(X: torch.Tensor, y: torch.Tensor, model: nn.Module, ensemble_vectors: torch.Tensor): # -> torch.Tensor:
    """
    Calculates nonconformity scores for Bayesian ensemble predictions.
    Score is -log(averaged_softmax_probability_of_true_label).

    Args:
        X (torch.Tensor): Input features.
        y (torch.Tensor): True labels.
        model (nn.Module): An instance of the neural network model (used for ensemble_predict).
        ensemble_vectors (torch.Tensor): Ensemble of model parameters.

    Returns:
        torch.Tensor: Vector of nonconformity scores.
    """
    # Get averaged softmax probabilities from the ensemble
    ens_pred = ensemble_predict(X, ensemble_vectors, model)

    # Select probabilities corresponding to the true labels
    true_label_probs = ens_pred[torch.arange(len(y)), y]

    # Calculate nonconformity score: -log(P_ensemble(true_label)) (Eq. 15 in main article)
    return -torch.log(true_label_probs)

def nonconformity_frq_giq(X: torch.Tensor, y: torch.Tensor, model: nn.Module): # -> torch.Tensor:
    """
    Calculates nonconformity scores for Frequentist (ML/MAP) predictions using
    Generalized Inverse Quantile (GIQ) method.
    The score is the cumulative probability mass until the true label is reached
    when classes are sorted by predicted probability (descending).

    Args:
        X (torch.Tensor): Input features.
        y (torch.Tensor): True labels.
        model (nn.Module): Trained neural network model.

    Returns:
        torch.Tensor: Vector of nonconformity scores (cumulative probabilities).
    """
    m_prob_y_pred = torch.nn.functional.softmax(model(X), dim=1)

    # Sort probabilities in descending order and get their original indices
    m_sorted_probs, m_sorted_indices = torch.sort(m_prob_y_pred, dim=1, descending=True)

    v_NC = torch.zeros(len(y), dtype=torch.float)

    for i_sample in range(len(y)):
        cumulative_prob = 0.0
        # Iterate through sorted classes until the true label is found
        for i_class_rank in range(m_sorted_probs.shape[1]): # Iterate through all classes if needed
            cumulative_prob += m_sorted_probs[i_sample, i_class_rank]
            # Check if the current class in the sorted list is the true label
            if m_sorted_indices[i_sample, i_class_rank] == y[i_sample]:
                break # Stop accumulating once the true label is found
        v_NC[i_sample] = cumulative_prob

    return v_NC

def nonconformity_bay_giq(X: torch.Tensor, y: torch.Tensor, model: nn.Module, ensemble_vectors: torch.Tensor): # -> torch.Tensor:
    """
    Calculates nonconformity scores for Bayesian ensemble predictions using
    Generalized Inverse Quantile (GIQ) method.
    The score is the cumulative probability mass until the true label is reached
    when classes are sorted by predicted probability (descending), using ensemble predictions.

    Args:
        X (torch.Tensor): Input features.
        y (torch.Tensor): True labels.
        model (nn.Module): An instance of the neural network model (used for ensemble_predict).
        ensemble_vectors (torch.Tensor): Ensemble of model parameters.

    Returns:
        torch.Tensor: Vector of nonconformity scores (cumulative probabilities).
    """
    m_prob_y_pred = ensemble_predict(X, ensemble_vectors, model)

    # Sort probabilities in descending order and get their original indices
    m_sorted_probs, m_sorted_indices = torch.sort(m_prob_y_pred, dim=1, descending=True)

    v_NC = torch.zeros(len(y), dtype=torch.float)

    for i_sample in range(len(y)):
        cumulative_prob = 0.0
        # Iterate through sorted classes until the true label is found
        for i_class_rank in range(m_sorted_probs.shape[1]):
            cumulative_prob += m_sorted_probs[i_sample, i_class_rank]
            if m_sorted_indices[i_sample, i_class_rank] == y[i_sample]:
                break
        v_NC[i_sample] = cumulative_prob

    return v_NC

## 7. Quantile Functions
def quantile_from_top(vec: torch.Tensor, alpha: float): # -> torch.Tensor:
    """
    Calculates the empirical quantile from the top (1-alpha quantile).
    Used for Conformal Prediction threshold.

    Args:
        vec (torch.Tensor): Input vector of scores.
        alpha (float): Miscoverage level (e.g., 0.1 for 90% coverage).

    Returns:
        torch.Tensor: The calculated quantile value.
    """
    # Append infinity to the vector to handle edge cases in quantile calculation
    torch_inf = torch.tensor(torch.inf).unsqueeze(0).to(vec.device) # Ensure inf is on same device as vec
    sorted_vec, _ = torch.sort(torch.cat((vec, torch_inf)))

    # Calculate the index for the quantile.
    # np.ceil((1-alpha)*(len(vec)+1)-1) corresponds to the rank in a 0-indexed sorted array.
    # This is equivalent to the original author's (1-alpha)*(N+1) rule.
    quantile_idx = int(np.ceil((1 - alpha) * (len(vec) + 1) - 1))

    return sorted_vec[quantile_idx]

def quantile_from_btm(vec: torch.Tensor, alpha: float): # -> torch.Tensor:
    """
    Calculates the empirical quantile from the bottom (alpha quantile).
    This is equivalent to finding the (1-alpha) quantile of the negative values.

    Args:
        vec (torch.Tensor): Input vector.
        alpha (float): Quantile level (e.g., 0.1).

    Returns:
        torch.Tensor: The calculated quantile value.
    """
    return -quantile_from_top(-vec, alpha)

# 8. Conformal Prediction Evaluation Functions

def vb__covrg_and_ineff(m_NC_prs: torch.Tensor, v_NC_val: torch.Tensor, y_te: torch.Tensor, alpha: float): # -> tuple[torch.Tensor, torch.Tensor]:
    """
    Calculates coverage for Validation-Based Conformal Prediction (VB-CP).

    Args:
        m_NC_prs (torch.Tensor): Prospective NC scores for all test samples and all possible labels.
                                 Shape [N_te, num_classes].
        v_NC_val (torch.Tensor): Nonconformity scores from the validation set. Shape [N_val].
        y_te (torch.Tensor): True labels for test samples.
        alpha (float): Miscoverage level.

    Returns:
        tuple[torch.Tensor, torch.Tensor]:
            - coverage (torch.Tensor): Overall empirical coverage.
            - coverage_per_label (torch.Tensor): Empirical coverage per true label.
    """
    # Calculate the quantile from the validation NC scores
    quan_val = quantile_from_top(v_NC_val, alpha) # Scalar threshold

    # Determine which labels are included in the prediction set for each test sample
    # m_is_prs_in_pred_set is a Boolean matrix [N_te, num_classes] (1 if included, 0 if not)
    m_is_prs_in_pred_set = (m_NC_prs <= quan_val).float()

    # Check if the true label (y_te) for each test sample is included in its prediction set
    v_is_y_te_in_pred = m_is_prs_in_pred_set[torch.arange(len(y_te)), y_te]

    # Calculate overall coverage (mean of whether true label was included)
    covrg = v_is_y_te_in_pred.mean()

    num_classes = m_is_prs_in_pred_set.shape[1]

    v_covrg_labels = torch.zeros(num_classes, dtype=torch.float)
    # v_ineff_labels is removed as per requirement

    # Calculate coverage per true label
    for i_y in torch.arange(num_classes):
        # Find indices of test samples that have current true label i_y
        indices_i_y = (y_te == i_y).nonzero(as_tuple=True)[0]
        if indices_i_y.numel() > 0: # Ensure there are samples for this label
            # Coverage for this label: mean of inclusion of true label for relevant samples
            v_covrg_labels[i_y] = m_is_prs_in_pred_set[indices_i_y, y_te[indices_i_y]].mean()
        else: # If no samples for this label, set to NaN
            v_covrg_labels[i_y] = float('nan')

    # Overall inefficiency calculation is removed
    # ineff = m_is_prs_in_pred_set.sum(dim=1).mean()

    # Return only coverage metrics
    return covrg, v_covrg_labels


def cv_covrg_and_ineff(m_NC_prs: torch.Tensor, v_NC: torch.Tensor, y_te: torch.Tensor, alpha: float): # -> tuple[torch.Tensor, torch.Tensor]:
    """
    Calculates coverage for CV-CP.
    Inefficiency calculations are removed.

    Args:
        m_NC_prs (torch.Tensor): Prospective NC scores for test samples.
                                 Shape: [N_te, num_classes, N_calib] (N_calib is total number of calibration samples).
        v_NC (torch.Tensor): All N (or N_calib) nonconformity scores from the calibration process. Shape [N_calib].
        y_te (torch.Tensor): True labels for test samples.
        alpha (float): Miscoverage level.

    Returns:
        tuple[torch.Tensor, torch.Tensor]:
            - coverage (torch.Tensor): Overall empirical coverage.
            - coverage_per_label (torch.Tensor): Empirical coverage per true label.
    """
    N_calib_total = len(v_NC)
    # th: Threshold for inclusion (based on (1-alpha)*(N_calib+1) from theory)
    th = (1 - alpha) * (N_calib_total + 1)

    # m_is_prs_in_pred_set: Boolean matrix [N_te, num_classes] indicating if a label is in the prediction set.
    # If this count (calib scores LARGER than prospective NC score) is less than `th`, the label is included. (Eq. 18)
    m_is_prs_in_pred_set = ((m_NC_prs > v_NC.view(1, 1, -1)).float().sum(dim=2) < th).float()

    # Check if the true label for each test sample is included in its prediction set
    v_is_y_te_in_pred = m_is_prs_in_pred_set[torch.arange(len(y_te)), y_te]

    # Calculate overall coverage
    covrg = v_is_y_te_in_pred.mean()

    num_classes = m_is_prs_in_pred_set.shape[1]

    v_covrg_labels = torch.zeros(num_classes, dtype=torch.float)
    # v_ineff_labels is removed

    # Calculate coverage per true label
    for i_y in torch.arange(num_classes):
        indices_i_y = (y_te == i_y).nonzero(as_tuple=True)[0]
        if indices_i_y.numel() > 0:
            v_covrg_labels[i_y] = m_is_prs_in_pred_set[indices_i_y, y_te[indices_i_y]].mean()
        else:
            v_covrg_labels[i_y] = float('nan')

    # Overall inefficiency calculation is removed
    # ineff = m_is_prs_in_pred_set.sum(dim=1).mean()

    # Return only coverage metrics
    return covrg, v_covrg_labels


def kcv_covrg_and_ineff(m_NC_prs: torch.Tensor, v_NC: torch.Tensor, y_te: torch.Tensor, alpha: float): # -> tuple[torch.Tensor, torch.Tensor]:
    """
    Calculates coverage for K-CV-CP.

    Args:
        m_NC_prs (torch.Tensor): Prospective NC scores for test samples.
                                 Shape: [N_te, num_classes, N_total_calibration_samples].
        v_NC (torch.Tensor): All N (or N_total_calibration_samples) nonconformity scores from the K-fold process. Shape [N_total_calibration_samples].
        y_te (torch.Tensor): True labels for test samples.
        alpha (float): Miscoverage level.

    Returns:
        tuple[torch.Tensor, torch.Tensor]:
            - coverage (torch.Tensor): Overall empirical coverage.
            - coverage_per_label (torch.Tensor): Empirical coverage per true label.
    """
    # The logic is identical to jkp_covrg_and_ineff for prediction set formation.
    # The distinction between KFP and JKP lies in how m_NC_prs and v_NC are generated.

    N_calib_total = len(v_NC)
    th = (1 - alpha) * (N_calib_total + 1)

    # If this count (calib scores LARGER than prospective NC score) is less than `th`, the label is included. (Eq. 18)
    m_is_prs_in_pred_set = ((m_NC_prs > v_NC.view(1, 1, -1)).float().sum(dim=2) < th).float()

    v_is_y_te_in_pred = m_is_prs_in_pred_set[torch.arange(len(y_te)), y_te]

    covrg = v_is_y_te_in_pred.mean()

    num_classes = m_is_prs_in_pred_set.shape[1]

    v_covrg_labels = torch.zeros(num_classes, dtype=torch.float)
    # v_ineff_labels is removed

    for i_y in torch.arange(num_classes):
        indices_i_y = (y_te == i_y).nonzero(as_tuple=True)[0]
        if indices_i_y.numel() > 0:
            v_covrg_labels[i_y] = m_is_prs_in_pred_set[indices_i_y, y_te[indices_i_y]].mean()
        else:
            v_covrg_labels[i_y] = float('nan')

    # Overall inefficiency calculation is removed
    # ineff = m_is_prs_in_pred_set.sum(dim=1).mean()

    # Return only coverage metrics
    return covrg, v_covrg_labels

In [ ]:
from tqdm import tqdm
import numpy as np
from scipy.io import loadmat, savemat
import torch
import datetime
import os
from copy import deepcopy


np.random.seed(12)
torch.manual_seed(12)


In [ ]:
# Parâmetros de simulação (definidos diretamente, como se viessem de argparse)
FIXED_SNR_DB = -30 # Signal to noise ratio per one RX antenna
FIXED_MOD_KEY = '8APSK' # Modulation key
FIXED_I_GAMMA = 8 # Index for regularization coeff gamma (from v_gamma_options below)
FIXED_GD_LR = 2e-1 # Learning rate while GDing
FIXED_GD_NUM_ITERS = 120 # Number of GD iterations (for ML and MAP)
FIXED_LMC_LR_INIT = 2e-1 # Initial Learning rate while LMC
FIXED_LMC_LR_DECAYING = 1.0 # Learning rate decaying factor over iterations
FIXED_LMC_BURN_IN = 100 # Total number of burn in first discarded model parameters while LMCing
FIXED_ENSEMBLE_SIZE = 20 # Ensemble prediction size for Bayesian methods
FIXED_NUM_SIM = 50 # Number of independent simulation runs
FIXED_COMPUTE_HESSIAN = 0 # {0: all zero hessian = faster = nonsense, 1: model-based using external network, 2: grad(grad) model agnostic}
COMPUTE_BAYES = True

In [ ]:
ALPHA = 0.1 # to 1-α
K_FOLDS = 4
N_FIXED = 24 # Fixed size of the training dataset (N)
# Número de amostras de teste (N_te original)
# LAST_LAYER_DIM virá de get8APSK().order
MOD_CONST_PARAMS = get8APSK()
LAST_LAYER_DIM = MOD_CONST_PARAMS['order'] # Number of classes (8 for 8APSK)
del MOD_CONST_PARAMS # Remover variável temporária
N_TEST_SAMPLES = LAST_LAYER_DIM * 12 # Number of test points (e.g., 8 * 12 = 96)


In [ ]:
# Miscoverage level for cross-validation-based methods
ALPHA_CVB = alpha_index

# Definir o espaço de gamma para regularização MAP
V_GAMMA_OPTIONS = [1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 1e-1, 2e-1, 5e-1, 1e0, 2e0, 5e0]
GAMMA_REG_COEFF = V_GAMMA_OPTIONS[FIXED_I_GAMMA] # Selected gamma based on index

# Definir índices para algoritmos (para indexar arrays de resultados)
I_ML = 0
I_MAP = 1
I_HES = 2
I_LMC = 3
ALG_STR_NAMES = ['frequentist ML', 'frequentist MAP', 'Bayesian using Hessian', 'Bayesian using LMC']

# Calcular o número total de iterações LMC
LMC_NUM_ITERS = FIXED_LMC_BURN_IN + FIXED_ENSEMBLE_SIZE - 1

# Definir dimensões da rede neural
FIRST_LAYER_DIM = 2 # I and Q components
V_LAYERS = [FIRST_LAYER_DIM, 5, 15, 15, LAST_LAYER_DIM] # Architecture: [Input, Hidden1, Hidden2, Hidden3, Output]

In [ ]:
M_ALLOC_2D_GLOBAL = np.empty((len(ALG_STR_NAMES), FIXED_NUM_SIM)) # Para cobertura geral (Algoritmo x Simulação)
M_ALLOC_3D_LABELS = np.empty((len(ALG_STR_NAMES), FIXED_NUM_SIM, LAST_LAYER_DIM)) # Para cobertura por label (Algoritmo x Simulação x Classe)

# Dicionários para armazenar resultados de COBERTURA
s_vb = {'covrg': M_ALLOC_2D_GLOBAL.copy(), 'covrg_labels': M_ALLOC_3D_LABELS.copy()}
s_jkp = {'covrg': M_ALLOC_2D_GLOBAL.copy(), 'covrg_labels': M_ALLOC_3D_LABELS.copy()}
s_kfp = {'covrg': M_ALLOC_2D_GLOBAL.copy(), 'covrg_labels': M_ALLOC_3D_LABELS.copy()}

# Históricos de perdas de treinamento/teste (ajustando dimensões para o N fixo)
# A dimensão do meio `1` é para o N_FIXED (já que não há loop sobre N, é sempre o primeiro/único N)
M_LOSS_TR_VB_ML = torch.zeros((FIXED_NUM_SIM, 1, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_VB_ML = torch.zeros((FIXED_NUM_SIM, 1, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TR_VB_MAP = torch.zeros((FIXED_NUM_SIM, 1, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_VB_MAP = torch.zeros((FIXED_NUM_SIM, 1, FIXED_GD_NUM_ITERS), dtype=torch.float)

# Para JKP, a dimensão é FIXED_NUM_SIM x N_FIXED (um modelo por LOO) x GD_NUM_ITERS
M_LOSS_TR_JKP_ML = torch.zeros((FIXED_NUM_SIM, N_FIXED, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_JKP_ML = torch.zeros((FIXED_NUM_SIM, N_FIXED, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TR_JKP_MAP = torch.zeros((FIXED_NUM_SIM, N_FIXED, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_JKP_MAP = torch.zeros((FIXED_NUM_SIM, N_FIXED, FIXED_GD_NUM_ITERS), dtype=torch.float)

# Para KFP, a dimensão é FIXED_NUM_SIM x K_FOLDS (um modelo por LFO) x GD_NUM_ITERS
M_LOSS_TR_KFP_ML = torch.zeros((FIXED_NUM_SIM, K_FOLDS, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_KFP_ML = torch.zeros((FIXED_NUM_SIM, K_FOLDS, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TR_KFP_MAP = torch.zeros((FIXED_NUM_SIM, K_FOLDS, FIXED_GD_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_KFP_MAP = torch.zeros((FIXED_NUM_SIM, K_FOLDS, FIXED_GD_NUM_ITERS), dtype=torch.float)

# Perdas LMC (ajustando dimensões)
M_LOSS_TR_VB_LMC = torch.zeros((FIXED_NUM_SIM, 1, LMC_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_VB_LMC = torch.zeros((FIXED_NUM_SIM, 1, LMC_NUM_ITERS), dtype=torch.float)
M_LOSS_TR_JKP_LMC = torch.zeros((FIXED_NUM_SIM, N_FIXED, LMC_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_JKP_LMC = torch.zeros((FIXED_NUM_SIM, N_FIXED, LMC_NUM_ITERS), dtype=torch.float)
M_LOSS_TR_KFP_LMC = torch.zeros((FIXED_NUM_SIM, K_FOLDS, LMC_NUM_ITERS), dtype=torch.float)
M_LOSS_TE_KFP_LMC = torch.zeros((FIXED_NUM_SIM, K_FOLDS, LMC_NUM_ITERS), dtype=torch.float)

# PARTE PRINCIPAL DA SIMULAÇÃO


In [ ]:
# Definir o caminho para salvar os resultados no Google Drive
PATH_OF_RUN = '/content/drive/MyDrive/resultados_simulacao/' # OU um caminho mais específico, ex: '/content/drive/MyDrive/MeuProjeto/resultados/'
try:
    os.stat(PATH_OF_RUN)
except:
    os.makedirs(PATH_OF_RUN) # Use os.makedirs para criar pastas aninhadas se elas não existirem

In [ ]:
MOD_CONST_PARAMS = get8APSK()

In [ ]:
start_time_total = datetime.datetime.now()
for i_s in tqdm(range(FIXED_NUM_SIM), desc="Total Simulations"):

    # Gerar novo estado de canal (psi, epsilon, delta) para esta rodada de simulação
    current_channel_state = channel_states()

    # Gerar dataset completo (X, y) para treinamento (N_FIXED) e teste (N_TEST_SAMPLES)
    # Estes datasets são fixos para esta rodada de simulação
    D_FULL_X, D_FULL_Y = simulate_channel_application(
        num_samples=N_FIXED,
        b_enforce_pattern=False, # Use padrão aleatório
        b_noise_free=False,
        d_setting={'snr_dB': FIXED_SNR_DB}, # SNR para a simulação
        channel_state=current_channel_state,
        mod_constellation_params=MOD_CONST_PARAMS
    )

    D_TEST_X, D_TEST_Y = simulate_channel_application(
        num_samples=N_TEST_SAMPLES,
        b_enforce_pattern=False, # Use padrão aleatório
        b_noise_free=False,
        d_setting={'snr_dB': FIXED_SNR_DB},
        channel_state=current_channel_state,
        mod_constellation_params=MOD_CONST_PARAMS
    )

    # Inicializar a rede neural (mesma inicialização para todos os métodos em cada simulação)
    net_init = FcReluDnn(V_LAYERS)
    gd_init_sd = net_init.state_dict() # Obter o estado inicial dos parâmetros para resetar modelos

    # N é fixo, correspondendo a N_FIXED
    N = N_FIXED # Para compatibilidade com nomes de variáveis na lógica

    # --- VB-CP (Validation-Based Conformal Prediction) ---
    print(f"Sim {i_s+1}/{FIXED_NUM_SIM} - VB-CP")
    # Divide D_FULL_X, D_FULL_Y em treinamento e validação para VB-CP
    N_TR_VB = int(N / 2) # Número de pontos para treinamento
    N_VAL_VB = N - N_TR_VB # Número de pontos para validação

    # D_TR_VB e D_VAL_VB são tuplas (X, y)
    (D_TR_VB_X, D_TR_VB_Y), (D_VAL_VB_X, D_VAL_VB_Y) = split_data_into_subsets(D_FULL_X, D_FULL_Y, N_TR_VB, shuffle=True)

    # Inicializar modelos para VB-CP
    model_vb_ml = deepcopy(net_init) # Modelo para Frequentist ML
    model_vb_map = deepcopy(net_init) # Modelo para Frequentist MAP
    model_vb_ens = deepcopy(net_init) # Modelo para Ensemble (Bayesiano)

    # Treinar modelo ML para VB-CP
    ml_loss_tr, ml_loss_te = fitting_erm_ml__gd(
        model=model_vb_ml, D_X=D_TR_VB_X, D_y=D_TR_VB_Y, D_te_X=D_TEST_X, D_te_y=D_TEST_Y,
        gd_init_sd=gd_init_sd, gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS
    )
    M_LOSS_TR_VB_ML[i_s, 0, :] = ml_loss_tr
    M_LOSS_TE_VB_ML[i_s, 0, :] = ml_loss_te

    # Treinar modelo MAP e gerar ensembles Bayesianos para VB-CP
    map_loss_tr, map_loss_te, lmc_loss_tr, lmc_loss_te, \
    ens_vb_hes, ens_vb_fim, ens_vb_lmc = fitting_erm_map_gd(
        model=model_vb_map, D_X_training=D_TR_VB_X, D_y_training=D_TR_VB_Y,
        D_test_X=D_TEST_X, D_test_y=D_TEST_Y, gd_init_sd=gd_init_sd,
        gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS, gamma=GAMMA_REG_COEFF,
        ensemble_size=FIXED_ENSEMBLE_SIZE, compute_hessian=FIXED_COMPUTE_HESSIAN,
        lmc_burn_in=FIXED_LMC_BURN_IN, lmc_lr_init=FIXED_LMC_LR_INIT,
        lmc_lr_decaying=FIXED_LMC_LR_DECAYING, compute_bays=COMPUTE_BAYES
    )
    M_LOSS_TR_VB_MAP[i_s, 0, :] = map_loss_tr
    M_LOSS_TE_VB_MAP[i_s, 0, :] = map_loss_te
    M_LOSS_TR_VB_LMC[i_s, 0, :] = lmc_loss_tr
    M_LOSS_TE_VB_LMC[i_s, 0, :] = lmc_loss_te

    # Calcular NC scores para VB-CP (no conjunto de VALIDAÇÃO)
    v_NC_vb_ml = nonconformity_frq(D_VAL_VB_X, D_VAL_VB_Y, model_vb_ml) # Eq. 15
    v_NC_vb_map = nonconformity_frq(D_VAL_VB_X, D_VAL_VB_Y, model_vb_map) # Eq. 15
    v_NC_vb_hes = nonconformity_bay(D_VAL_VB_X, D_VAL_VB_Y, model_vb_ens, ens_vb_hes) # Eq. 15
    v_NC_vb_lmc = nonconformity_bay(D_VAL_VB_X, D_VAL_VB_Y, model_vb_ens, ens_vb_lmc) # Eq. 15

    # Preparar X_pairs e y_pairs para cálculo de NC scores prospectivos (para cada X_test e cada Y_prime)
    Y_PRIME = torch.tensor(range(LAST_LAYER_DIM)) # Possible output labels
    X_PAIRS = D_TEST_X.repeat_interleave(len(Y_PRIME), dim=0) # X_pairs original
    Y_PAIRS = Y_PRIME.repeat(D_TEST_X.shape[0]) # y_pairs original

    # Calcular NC scores prospectivos para VB-CP (no conjunto de TESTE)
    m_NC_vb_ml_prs = nonconformity_frq(X_PAIRS, Y_PAIRS, model_vb_ml).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
    m_NC_vb_map_prs = nonconformity_frq(X_PAIRS, Y_PAIRS, model_vb_map).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
    m_NC_vb_hes_prs = nonconformity_bay(X_PAIRS, Y_PAIRS, model_vb_ens, ens_vb_hes).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
    m_NC_vb_lmc_prs = nonconformity_bay(X_PAIRS, Y_PAIRS, model_vb_ens, ens_vb_lmc).view(N_TEST_SAMPLES, LAST_LAYER_DIM)

    # Armazenar resultados de cobertura para VB-CP (sem ineficiência)
    s_vb['covrg'][I_ML, i_s], s_vb['covrg_labels'][I_ML, i_s, :] = vb__covrg_and_ineff(m_NC_vb_ml_prs, v_NC_vb_ml, D_TEST_Y, ALPHA)
    s_vb['covrg'][I_MAP, i_s], s_vb['covrg_labels'][I_MAP, i_s, :] = vb__covrg_and_ineff(m_NC_vb_map_prs, v_NC_vb_map, D_TEST_Y, ALPHA)
    s_vb['covrg'][I_HES, i_s], s_vb['covrg_labels'][I_HES, i_s, :] = vb__covrg_and_ineff(m_NC_vb_hes_prs, v_NC_vb_hes, D_TEST_Y, ALPHA)
    s_vb['covrg'][I_LMC, i_s], s_vb['covrg_labels'][I_LMC, i_s, :] = vb__covrg_and_ineff(m_NC_vb_lmc_prs, v_NC_vb_lmc, D_TEST_Y, ALPHA)

    # --- JKP (Jackknife+ Conformal Prediction) or CV-CP ---
    print(f"Sim {i_s+1}/{FIXED_NUM_SIM} - JKP")
    # Treina N modelos (Leave-One-Out) para JKP
    l_model_jkp_ml = [] # Lista de modelos ML para JKP
    l_model_jkp_map = [] # Lista de modelos MAP para JKP
    l_ens_jkp_hes = [] # Lista de ensembles Hessian para JKP
    l_ens_jkp_lmc = [] # Lista de ensembles LMC para JKP

    # Tensors para armazenar NC scores de calibração para JKP
    v_NC_jkp_ml = torch.zeros(N, dtype=torch.float)
    v_NC_jkp_map = torch.zeros(N, dtype=torch.float)
    v_NC_jkp_hes = torch.zeros(N, dtype=torch.float)
    v_NC_jkp_lmc = torch.zeros(N, dtype=torch.float)

    # Tensors para armazenar NC scores prospectivos (teste vs calibração) para JKP
    # Dimensão [N_te, num_classes, N_total_calibration_samples]
    m_NC_jkp_ml_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_jkp_map_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_jkp_hes_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_jkp_lmc_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)

    # Loop para treinar N modelos (LOO) e coletar NC scores para JKP
    for i_loo in tqdm(range(N), desc="JKP LOO Training"): # N is D_FULL_X's length
        # Criar dataset Leave-One-Out
        D_LOO_X, D_LOO_Y = leave_one_out_data(D_FULL_X, D_FULL_Y, i_loo)

        # Inicializar modelos para esta iteração LOO
        model_jkp_ml = deepcopy(net_init)
        model_jkp_map = deepcopy(net_init)
        model_jkp_ens = deepcopy(net_init) # Modelo para ensemble, parâmetros carregados por torch.nn.utils.vector_to_parameters

        # Treinar ML model para JKP
        ml_loss_tr, ml_loss_te = fitting_erm_ml__gd(
            model=model_jkp_ml, D_X=D_LOO_X, D_y=D_LOO_Y, D_te_X=D_TEST_X, D_te_y=D_TEST_Y,
            gd_init_sd=gd_init_sd, gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS
        )
        M_LOSS_TR_JKP_ML[i_s, i_loo, :] = ml_loss_tr
        M_LOSS_TE_JKP_ML[i_s, i_loo, :] = ml_loss_te

        # Treinar MAP model e gerar ensembles para JKP
        map_loss_tr, map_loss_te, lmc_loss_tr, lmc_loss_te, \
        ens_jkp_hes, ens_jkp_fim, ens_jkp_lmc = fitting_erm_map_gd(
            model=model_jkp_map, D_X_training=D_LOO_X, D_y_training=D_LOO_Y,
            D_test_X=D_TEST_X, D_test_y=D_TEST_Y, gd_init_sd=gd_init_sd,
            gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS, gamma=GAMMA_REG_COEFF,
            ensemble_size=FIXED_ENSEMBLE_SIZE, compute_hessian=FIXED_COMPUTE_HESSIAN,
            lmc_burn_in=FIXED_LMC_BURN_IN, lmc_lr_init=FIXED_LMC_LR_INIT,
            lmc_lr_decaying=FIXED_LMC_LR_DECAYING, compute_bays=COMPUTE_BAYES
        )
        M_LOSS_TR_JKP_MAP[i_s, i_loo, :] = map_loss_tr
        M_LOSS_TE_JKP_MAP[i_s, i_loo, :] = map_loss_te
        M_LOSS_TR_JKP_LMC[i_s, i_loo, :] = lmc_loss_tr
        M_LOSS_TE_JKP_LMC[i_s, i_loo, :] = lmc_loss_te

        # Armazenar modelos e ensembles em listas para uso posterior
        l_model_jkp_ml.append(model_jkp_ml)
        l_model_jkp_map.append(model_jkp_map)
        l_ens_jkp_hes.append(ens_jkp_hes)
        l_ens_jkp_lmc.append(ens_jkp_lmc)

        # Calcular NC scores de calibração (do ponto "left out") para JKP
        x_i_loo = D_FULL_X[i_loo, :].view(1, -1) # Ponto removido (calibração)
        y_i_loo = D_FULL_Y[i_loo].view(-1) # Rótulo do ponto removido

        v_NC_jkp_ml[i_loo] = nonconformity_frq(x_i_loo, y_i_loo, model_jkp_ml) # Eq. 15
        v_NC_jkp_map[i_loo] = nonconformity_frq(x_i_loo, y_i_loo, model_jkp_map) # Eq. 15
        v_NC_jkp_hes[i_loo] = nonconformity_bay(x_i_loo, y_i_loo, model_jkp_ens, ens_jkp_hes) # Eq. 15
        v_NC_jkp_lmc[i_loo] = nonconformity_bay(x_i_loo, y_i_loo, model_jkp_ens, ens_jkp_lmc) # Eq. 15

        # Calcular NC scores prospectivos (teste vs calibração) para JKP
        # m_NC_jkp_ml_prs[:,:,i_loo] armazena o NC score prospectivo calculado com o modelo
        # treinado SEM essa amostra `i_loo`.
        m_NC_jkp_ml_prs[:, :, i_loo] = nonconformity_frq(X_PAIRS, Y_PAIRS, l_model_jkp_ml[i_loo]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_jkp_map_prs[:, :, i_loo] = nonconformity_frq(X_PAIRS, Y_PAIRS, l_model_jkp_map[i_loo]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_jkp_hes_prs[:, :, i_loo] = nonconformity_bay(X_PAIRS, Y_PAIRS, model_jkp_ens, l_ens_jkp_hes[i_loo]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_jkp_lmc_prs[:, :, i_loo] = nonconformity_bay(X_PAIRS, Y_PAIRS, model_jkp_ens, l_ens_jkp_lmc[i_loo]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)

    # Armazenar resultados de cobertura para JKP (sem ineficiência)
    s_jkp['covrg'][I_ML, i_s], s_jkp['covrg_labels'][I_ML, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_ml_prs, v_NC_jkp_ml, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_jkp['covrg'][I_MAP, i_s], s_jkp['covrg_labels'][I_MAP, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_map_prs, v_NC_jkp_map, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_jkp['covrg'][I_HES, i_s], s_jkp['covrg_labels'][I_HES, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_hes_prs, v_NC_jkp_hes, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_jkp['covrg'][I_LMC, i_s], s_jkp['covrg_labels'][I_LMC, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_lmc_prs, v_NC_jkp_lmc, D_TEST_Y, ALPHA_CVB) # Eq. 18

    # --- KFP (K-fold Conformal Prediction) ---
    print(f"Sim {i_s+1}/{FIXED_NUM_SIM} - KFP")
    # Treina K modelos (Leave-Fold-Out) para KFP
    N_OVER_K = round(N / K_FOLDS)
    assert(N_OVER_K * K_FOLDS == N) # Assegurar divisibilidade perfeita para K-folds

    l_model_kfp_ml = []
    l_model_kfp_map = []
    l_ens_kfp_hes = []
    l_ens_kfp_lmc = []

    # Tensors para armazenar NC scores de calibração para KFP
    v_NC_kfp_ml = torch.zeros(N, dtype=torch.float)
    v_NC_kfp_map = torch.zeros(N, dtype=torch.float)
    v_NC_kfp_hes = torch.zeros(N, dtype=torch.float)
    v_NC_kfp_lmc = torch.zeros(N, dtype=torch.float)

    # Tensors para armazenar NC scores prospectivos (teste vs calibração) para KFP
    m_NC_kfp_ml_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_kfp_map_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_kfp_hes_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)
    m_NC_kfp_lmc_prs = torch.zeros((N_TEST_SAMPLES, LAST_LAYER_DIM, N), dtype=torch.float)

    # Loop para treinar K modelos (LFO) e coletar NC scores para KFP
    V_K_FOLDS_MAP = torch.arange(K_FOLDS).repeat_interleave(N_OVER_K) # Mapping for index to fold
    for k_idx in tqdm(range(K_FOLDS), desc="KFP LFO Training"):
        # Criar dataset Leave-Fold-Out
        D_LFO_X, D_LFO_Y = leave_fold_out_data(D_FULL_X, D_FULL_Y, k_idx, K_FOLDS)

        # Inicializar modelos para esta iteração LFO
        model_kfp_ml = deepcopy(net_init)
        model_kfp_map = deepcopy(net_init)
        model_kfp_ens = deepcopy(net_init)

        # Treinar ML model para KFP
        ml_loss_tr, ml_loss_te = fitting_erm_ml__gd(
            model=model_kfp_ml, D_X=D_LFO_X, D_y=D_LFO_Y, D_te_X=D_TEST_X, D_te_y=D_TEST_Y,
            gd_init_sd=gd_init_sd, gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS
        )
        M_LOSS_TR_KFP_ML[i_s, k_idx, :] = ml_loss_tr
        M_LOSS_TE_KFP_ML[i_s, k_idx, :] = ml_loss_te

        # Treinar MAP model e gerar ensembles para KFP
        map_loss_tr, map_loss_te, lmc_loss_tr, lmc_loss_te, \
        ens_kfp_hes, ens_kfp_fim, ens_kfp_lmc = fitting_erm_map_gd(
            model=model_kfp_map, D_X_training=D_LFO_X, D_y_training=D_LFO_Y,
            D_test_X=D_TEST_X, D_test_y=D_TEST_Y, gd_init_sd=gd_init_sd,
            gd_lr=FIXED_GD_LR, gd_num_iters=FIXED_GD_NUM_ITERS, gamma=GAMMA_REG_COEFF,
            ensemble_size=FIXED_ENSEMBLE_SIZE, compute_hessian=FIXED_COMPUTE_HESSIAN,
            lmc_burn_in=FIXED_LMC_BURN_IN, lmc_lr_init=FIXED_LMC_LR_INIT,
            lmc_lr_decaying=FIXED_LMC_LR_DECAYING, compute_bays=COMPUTE_BAYES
        )
        M_LOSS_TR_KFP_MAP[i_s, k_idx, :] = map_loss_tr
        M_LOSS_TE_KFP_MAP[i_s, k_idx, :] = map_loss_te
        M_LOSS_TR_KFP_LMC[i_s, k_idx, :] = lmc_loss_tr
        M_LOSS_TE_KFP_LMC[i_s, k_idx, :] = lmc_loss_te

        # Armazenar modelos e ensembles em listas para uso posterior
        l_model_kfp_ml.append(model_kfp_ml)
        l_model_kfp_map.append(model_kfp_map)
        l_ens_kfp_hes.append(ens_kfp_hes)
        l_ens_kfp_lmc.append(ens_kfp_lmc)

    # Calcular NC scores de calibração (do ponto "left out") para KFP
    for i_sample in tqdm(range(N), desc="KFP NC Score Calculation"): # Loop sobre todos os N pontos do D_FULL
        k_idx_for_sample = V_K_FOLDS_MAP[i_sample] # Determina a qual fold este sample pertence
        x_i_lfo = D_FULL_X[i_sample, :].view(1, -1) # O ponto de calibração
        y_i_lfo = D_FULL_Y[i_sample].view(-1) # O rótulo do ponto de calibração

        # Calcula o NC score para o ponto `x_i_lfo, y_i_lfo` usando o modelo *que não o viu* (i.e., do fold `k_idx_for_sample`)
        v_NC_kfp_ml[i_sample] = nonconformity_frq(x_i_lfo, y_i_lfo, l_model_kfp_ml[k_idx_for_sample]) # Eq. 15
        v_NC_kfp_map[i_sample] = nonconformity_frq(x_i_lfo, y_i_lfo, l_model_kfp_map[k_idx_for_sample]) # Eq. 15
        v_NC_kfp_hes[i_sample] = nonconformity_bay(x_i_lfo, y_i_lfo, model_kfp_ens, l_ens_kfp_hes[k_idx_for_sample]) # Eq. 15
        v_NC_kfp_lmc[i_sample] = nonconformity_bay(x_i_lfo, y_i_lfo, model_kfp_ens, l_ens_kfp_lmc[k_idx_for_sample]) # Eq. 15

        # Calcular NC scores prospectivos (teste vs calibração) para KFP
        m_NC_kfp_ml_prs[:, :, i_sample] = nonconformity_frq(X_PAIRS, Y_PAIRS, l_model_kfp_ml[k_idx_for_sample]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_kfp_map_prs[:, :, i_sample] = nonconformity_frq(X_PAIRS, Y_PAIRS, l_model_kfp_map[k_idx_for_sample]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_kfp_hes_prs[:, :, i_sample] = nonconformity_bay(X_PAIRS, Y_PAIRS, model_kfp_ens, l_ens_kfp_hes[k_idx_for_sample]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)
        m_NC_kfp_lmc_prs[:, :, i_sample] = nonconformity_bay(X_PAIRS, Y_PAIRS, model_kfp_ens, l_ens_kfp_lmc[k_idx_for_sample]).view(N_TEST_SAMPLES, LAST_LAYER_DIM)

    # Armazenar resultados de cobertura para KFP (sem ineficiência)
    s_kfp['covrg'][I_ML, i_s], s_kfp['covrg_labels'][I_ML, i_s, :] = kcv_covrg_and_ineff(m_NC_kfp_ml_prs, v_NC_kfp_ml, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_kfp['covrg'][I_MAP, i_s], s_kfp['covrg_labels'][I_MAP, i_s, :] = kcv_covrg_and_ineff(m_NC_kfp_map_prs, v_NC_kfp_map, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_kfp['covrg'][I_HES, i_s], s_kfp['covrg_labels'][I_HES, i_s, :] = kcv_covrg_and_ineff(m_NC_kfp_hes_prs, v_NC_kfp_hes, D_TEST_Y, ALPHA_CVB) # Eq. 18
    s_kfp['covrg'][I_LMC, i_s], s_kfp['covrg_labels'][I_LMC, i_s, :] = kcv_covrg_and_ineff(m_NC_kfp_lmc_prs, v_NC_kfp_lmc, D_TEST_Y, ALPHA_CVB) # Eq. 18


Total Simulations:   0%|          | 0/50 [00:00<?, ?it/s]

Sim 1/50 - VB-CP


/tmp/ipython-input-67-3539394095.py:89: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_vb['covrg'][I_ML, i_s], s_vb['covrg_labels'][I_ML, i_s, :] = vb__covrg_and_ineff(m_NC_vb_ml_prs, v_NC_vb_ml, D_TEST_Y, ALPHA)
/tmp/ipython-input-67-3539394095.py:90: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_vb['covrg'][I_MAP, i_s], s_vb['covrg_labels'][I_MAP, i_s, :] = vb__covrg_and_ineff(m_NC_vb_map_prs, v_NC_vb_map, D_TEST_Y, ALPHA)
/tmp/ipython-input-67-3539394095.py:91: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_vb['covrg'][I_HES, i_s], s_vb['covrg_labels'][I_HES, i_s, :] = vb__covrg_and_ineff(m_NC_vb_hes_prs, v_NC_

Sim 1/50 - JKP



JKP LOO Training: 100%|██████████| 24/24 [00:08<00:00,  2.81it/s]
/tmp/ipython-input-67-3539394095.py:172: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_jkp['covrg'][I_ML, i_s], s_jkp['covrg_labels'][I_ML, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_ml_prs, v_NC_jkp_ml, D_TEST_Y, ALPHA_CVB) # Eq. 18
/tmp/ipython-input-67-3539394095.py:173: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_jkp['covrg'][I_MAP, i_s], s_jkp['covrg_labels'][I_MAP, i_s, :] = cv_covrg_and_ineff(m_NC_jkp_map_prs, v_NC_jkp_map, D_TEST_Y, ALPHA_CVB) # Eq. 18
/tmp/ipython-input-67-3539394095.py:174: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  s_jkp

...


KFP LFO Training: 100%|██████████| 4/4 [00:01<00:00,  2.86it/s]

Total Simulations:  98%|█████████▊| 49/50 [09:15<00:11, 11.50s/it]

Sim 50/50 - VB-CP
Sim 50/50 - JKP



JKP LOO Training: 100%|██████████| 24/24 [00:08<00:00,  2.73it/s]


Sim 50/50 - KFP



KFP LFO Training: 100%|██████████| 4/4 [00:01<00:00,  2.68it/s]

Total Simulations: 100%|██████████| 50/50 [09:27<00:00, 11.34s/it]


In [ ]:
print(f"\n--- Resultados de Cobertura Média por Método ---\n SNR={FIXED_SNR_DB}, \D\={N_FIXED}")

# Para cada dicionário de resultados (VB-CP, JKP, KFP e suas variantes GIQ)
results_dicts = {
    "VB-CP (Log-Loss)": s_vb,
    "CV-CP (Log-Loss)": s_jkp,
    "K-CV-CP (Log-Loss)": s_kfp,
}

# I_ML, I_MAP, I_HES, I_FIM, I_LMC são os índices dos algoritmos
# ALG_STR_NAMES são os nomes dos algoritmos

for method_name, results_data in results_dicts.items():
    print(f"\n--- {method_name} ---")
    # Acessa o array de cobertura geral para este método/variante
    coverage_array = results_data['covrg'] # Shape: [len(ALG_STR_NAMES), FIXED_NUM_SIM]

    for alg_idx, alg_name in enumerate(ALG_STR_NAMES):
        # Calcula a média da cobertura sobre todas as rodadas de simulação para este algoritmo
        avg_coverage_for_alg = coverage_array[alg_idx, :].mean()
        # Calcula o desvio padrão da cobertura sobre todas as rodadas de simulação
        std_coverage_for_alg = coverage_array[alg_idx, :].std()

        print(f"   {alg_name}: Média = {avg_coverage_for_alg:.4f}, Desvio Padrão = {std_coverage_for_alg:.4f}")

# Exemplo de como acessar a cobertura por classe para um método e algoritmo específico
# (Descomente se quiser explorar mais a fundo, mas o loop acima já dá o geral)
# print("\n--- Exemplo: Cobertura por Classe para VB-CP (Log-Loss), Frequentist ML ---")
# # s_vb_['covrg_labels'] tem shape [len(ALG_STR_NAMES), FIXED_NUM_SIM, LAST_LAYER_DIM]
# coverage_labels_example = s_vb_['covrg_labels'][I_ML, :, :]
# # Média sobre as simulações para cada classe
# avg_coverage_per_class = np.nanmean(coverage_labels_example, axis=0) # Usa nanmean para ignorar NaNs
# # Desvio padrão sobre as simulações para cada classe
# std_coverage_per_class = np.nanstd(coverage_labels_example, axis=0)
# for class_idx in range(LAST_LAYER_DIM):
#     print(f"   Classe {class_idx}: Média = {avg_coverage_per_class[class_idx]:.4f}, Desvio Padrão = {std_coverage_per_class[class_idx]:.4f}")

# ==============================================================================
# FIM da Seção de Análise e Início da Seção de Salvamento
# ==============================================================================

# --- Salvar Resultados (usando scipy.io.savemat) ---
import scipy.io # Importar scipy.io para savemat (se ainda não estiver importado)
file_str = 'simulation_results_fixed_N.mat' # Nome do arquivo de saída
scipy.io.savemat(PATH_OF_RUN + file_str, {
    "dSetting": {'snr_dB': FIXED_SNR_DB, 'modKey': FIXED_MOD_KEY},
    "N_fixed": N_FIXED,
    "gamma": GAMMA_REG_COEFF,
    "num_sim": FIXED_NUM_SIM,
    "alg_str_names": ALG_STR_NAMES,
    "i_ml": I_ML, "i_map": I_MAP, "i_hes": I_HES, "i_lmc": I_LMC,
    "alpha": ALPHA, "alpha_cvb": ALPHA_CVB, "K_folds": K_FOLDS,
    "gd_num_iters": FIXED_GD_NUM_ITERS, "gd_lr": FIXED_GD_LR,
    "lmc_burn_in": FIXED_LMC_BURN_IN, "lmc_lr_init": FIXED_LMC_LR_INIT, "lmc_lr_decaying": FIXED_LMC_LR_DECAYING,
    "lmc_num_iters": LMC_NUM_ITERS,
    "ensemble_size": FIXED_ENSEMBLE_SIZE, "compute_hessian": FIXED_COMPUTE_HESSIAN,
    # Estruturas de resultados de cobertura
    "s_vb": s_vb,
    "s_jkp": s_jkp,
    "s_kfp": s_kfp,

    # Históricos de perda
    "m_loss_tr_vb_ml": M_LOSS_TR_VB_ML.numpy(), "m_loss_te_vb_ml": M_LOSS_TE_VB_ML.numpy(),
    "m_loss_tr_vb_map": M_LOSS_TR_VB_MAP.numpy(), "m_loss_te_vb_map": M_LOSS_TE_VB_MAP.numpy(),
    "m_loss_tr_jkp_ml": M_LOSS_TR_JKP_ML.numpy(), "m_loss_te_jkp_ml": M_LOSS_TE_JKP_ML.numpy(),
    "m_loss_tr_jkp_map": M_LOSS_TR_JKP_MAP.numpy(), "m_loss_te_jkp_map": M_LOSS_TE_JKP_MAP.numpy(),
    "m_loss_tr_kfp_ml": M_LOSS_TR_KFP_ML.numpy(), "m_loss_te_kfp_ml": M_LOSS_TE_KFP_ML.numpy(),
    "m_loss_tr_kfp_map": M_LOSS_TR_KFP_MAP.numpy(), "m_loss_te_kfp_map": M_LOSS_TE_KFP_MAP.numpy(),
    "m_loss_tr_vb_lmc": M_LOSS_TR_VB_LMC.numpy(), "m_loss_te_vb_lmc": M_LOSS_TE_VB_LMC.numpy(),
    "m_loss_tr_jkp_lmc": M_LOSS_TR_JKP_LMC.numpy(), "m_loss_te_jkp_lmc": M_LOSS_TE_JKP_LMC.numpy(),
    "m_loss_tr_kfp_lmc": M_LOSS_TR_KFP_LMC.numpy(), "m_loss_te_kfp_lmc": M_LOSS_TE_KFP_LMC.numpy(),
})
print(f'Results saved to: {PATH_OF_RUN + file_str}')

# --- Finalização ---
print('Simulation done. Total time:')
print(datetime.datetime.now() - start_time_total)


--- Resultados de Cobertura Média por Método ---
 SNR=-30, \D\=24

--- VB-CP (Log-Loss) ---
   frequentist ML: Média = 0.9396, Desvio Padrão = 0.0724
   frequentist MAP: Média = 0.9504, Desvio Padrão = 0.0460
   Bayesian using Hessian: Média = 0.9504, Desvio Padrão = 0.0460
   Bayesian using LMC: Média = 0.9371, Desvio Padrão = 0.0628

--- CV-CP (Log-Loss) ---
   frequentist ML: Média = 0.9712, Desvio Padrão = 0.0519
   frequentist MAP: Média = 0.9706, Desvio Padrão = 0.0445
   Bayesian using Hessian: Média = 0.9706, Desvio Padrão = 0.0445
   Bayesian using LMC: Média = 0.9754, Desvio Padrão = 0.0430

--- K-CV-CP (Log-Loss) ---
   frequentist ML: Média = 0.9792, Desvio Padrão = 0.0410
   frequentist MAP: Média = 0.9735, Desvio Padrão = 0.0385
   Bayesian using Hessian: Média = 0.9735, Desvio Padrão = 0.0385
   Bayesian using LMC: Média = 0.9881, Desvio Padrão = 0.0247
Results saved to: /content/drive/MyDrive/resultados_simulacao/simulation_results_fixed_N.mat
Simulation done. Total ti

In [ ]:
"""
#Opcional: Liberação de memória (mais relevante para GPUs)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
import gc
gc.collect()"""

'\n#Opcional: Liberação de memória (mais relevante para GPUs)\nif torch.cuda.is_available():\n    torch.cuda.empty_cache()\n    torch.cuda.ipc_collect()\nimport gc\ngc.collect()'